In [ ]:
%load_ext autoreload
%autoreload 2

: 

In [ ]:
from pathlib import Path
import os

notebook_dir = Path().resolve()
os.chdir(notebook_dir)

In [ ]:
from data_loading import standardise_month_jsons
standardise_month_jsons()

In [ ]:
from data_loading import load_month_data, generate_derived_features
import pandas as pd
data = load_month_data()
data_derived = generate_derived_features(data)

In [ ]:
def print_col_stats(df, col):
    print(f"Column: {col}")
    print(f"Data type: {df[col].dtype}")
    print(f"Missing values: {df[col].isnull().sum()}")
    print(f"Unique values: {df[col].nunique()}")
    print(f"\nValue counts:\n{df[col].value_counts()}\n\n\n\n")

In [ ]:
print_col_stats(data_derived, "recovery_ratio")
print_col_stats(data_derived, "supplier")

In [ ]:
data_derived['supplier']

In [ ]:
from visualisation import generate_plots, scatter_coloured, density_scatter, display_simple_model_with_lgbm_and_density_scatter
#generate_plots(data_derived)

In [ ]:
data_derived_without_supplied_m3 = data_derived.drop(columns=["supplied_m3"], inplace=False)

In [ ]:
interesting_combinations_no_supplied_target_recover_m3 = [
    ("process_time_mins", "start_hour", "recovery_ratio"),
    ("start_day_week", "process_time_mins", "recovery_ratio"),
    ("supplier", "start_hour", "recovery_ratio"),
    ("facility", "start_hour", "recovery_ratio"),
    ("month", "start_day_week", "recovery_ratio"),
    ("supplier", "facility", "recovery_ratio"),
    ("facility", "process_time_mins", "recovery_ratio"),
]

interesting_combinations_with_supplied_m3 = [
    ("supplier", "supplied_m3", "recovery_ratio"),
    ("supplier", "recovery_ratio", "supplied_m3"),
    ("supplied_m3", "recovery_ratio", "supplier"),
]


In [ ]:
for x_col, y_col, target in interesting_combinations_with_supplied_m3:
    density_scatter(data_derived, x_col, y_col, target, bw = 0.2, alpha=0.5)

In [ ]:
from statistical_comparision import analyze_condition_impact, compare_two_conditions, print_impact_summary, compare_two_columns, visualise_impact_summary


In [ ]:
# Compare one supplier vs all others
results = analyze_condition_impact(data_derived, "supplier", "Mary")
#print_impact_summary(results)

# Compare two specific suppliers
results = compare_two_conditions(data_derived, "supplier", "Mary", "Mary Anne")
print_impact_summary(results, alpha=0.9)

In [ ]:
results = compare_two_columns(data_derived, "supplied_m3", "recovery_ratio")
print_impact_summary(results)
visualise_impact_summary(results)
results = compare_two_columns(data_derived, "supplied_m3", "recovery_ratio")
print_impact_summary(results)
visualise_impact_summary(results)